### 1. Modelos

Los modelos son el núcleo de LangChain y se encargan de procesar texto. Puedes elegir entre diferentes modelos en función de tu necesidad:

- OpenAI: Ideal para tareas generales como generación de texto o respuestas a preguntas.
- LlamaCpp: Diseñado para implementaciones locales y entornos sin conexión, útil si necesitas un control completo sobre el modelo.

¿Cuándo usarlo?

- OpenAI: Cuando necesites alto rendimiento, fácil configuración y estés dispuesto a usar un servicio basado en la nube.
- LlamaCpp: Si prefieres trabajar localmente y manejar modelos personalizados sin depender de servicios externos.

In [ ]:
from langchain.llms import LlamaCpp, OpenAI 
import config

api_key = config.OPENAI_API_KEY  

# Modelo basado en OpenAI
llm_openai = OpenAI(model_name="text-davinci-003", openai_api_key=api_key)
respuesta_openai = llm_openai("Hola, ¿cómo estás?")
print(respuesta_openai)

# Modelo local con LlamaCpp
# llm_llama = LlamaCpp(model_path="./llamacpp/models/7B/ggml-model-q4_0.bin")
# respuesta_llama = llm_llama("Hola, ¿cómo estás?")
# print(respuesta_llama)


### 2. Modelos de Chat

Los modelos de chat, como ChatGPT, están diseñados específicamente para interacción conversacional. Estos modelos procesan mensajes estructurados para emular un diálogo fluido.
¿Por qué usar modelos de chat?
- Se ajustan mejor a flujos de conversación en tiempo real.
- Pueden manejar instrucciones específicas para cada rol (usuario, sistema, modelo).
- Son ideales para asistentes virtuales o chatbots.

Aplicación típica: Chatbots personalizados, sistemas de atención al cliente, aplicaciones que requieren respuestas contextuales.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

chatgpt = ChatOpenAI(openai_api_key=api_key)

# Enviando un mensaje al modelo
respuesta = chatgpt([HumanMessage(content="Hola, ¿cómo estás?")])
print(respuesta)


### 3. Prompts

Los prompts son la base para interactuar con modelos de lenguaje. Son instrucciones o preguntas que guían al modelo sobre qué respuesta se espera.
¿Cuándo usar prompts básicos?

- Para consultas simples donde el contexto no cambia con frecuencia.
- Cuando necesites respuestas específicas basadas en instrucciones claras.

In [ ]:
from langchain import PromptTemplate

template_basico = """Eres un asistente culinario que responde a preguntas de manera breve.
Pregunta: ¿Cuáles son los ingredientes para preparar {plato}?
Respuesta:"""

prompt_temp = PromptTemplate(input_variables=["plato"], template=template_basico)
promt_value = prompt_temp.format(plato="empanadas")
respuesta_openai = llm_openai(promt_value)
print(respuesta_openai)


### 4. ChatPromptTemplates

Los ChatPromptTemplates son versiones avanzadas de los prompts, diseñados específicamente para modelos de chat. Agregan roles claros (sistema, humano, modelo) para guiar mejor las interacciones.
¿Por qué usar ChatPromptTemplates?

- Cuando el modelo de chat necesita contexto adicional o roles claros.
- Para construir conversaciones estructuradas y coherentes.



Cómo Funciona la Integración

- Se crean mensajes individuales con SystemMessagePromptTemplate y HumanMessagePromptTemplate, definiendo el rol y el contenido de cada uno.
- Estos mensajes se ensamblan con ChatPromptTemplate para formar un prompt estructurado y apto para modelos de chat.
- Finalmente, este prompt combinado se envía al modelo para generar una respuesta contextualizada.

In [ ]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

# Definiendo contexto para el sistema
prompt_temp_sistema = PromptTemplate(
    template="Eres un asistente que recomienda alternativas {adjetivo} a productos.",
    input_variables=["adjetivo"],
)
template_sistema = SystemMessagePromptTemplate(prompt=prompt_temp_sistema)

# Prompt para el usuario
prompt_temp_humano = PromptTemplate(template="{texto}", input_variables=["texto"])
template_humano = HumanMessagePromptTemplate(prompt=prompt_temp_humano)

# Creación del chat template
chat_prompt = ChatPromptTemplate.from_messages([template_sistema, template_humano])

chat_promt_value = chat_prompt.format_prompt(adjetivo="económica", texto="iPad").to_messages()
respuesta_chat = chatgpt(chat_promt_value)
print(respuesta_chat)


### 5. Example Selector

Los Example Selectors permiten proporcionar ejemplos específicos al modelo para influir en las respuestas. Esto es útil cuando quieres que el modelo siga un formato o estilo específico.
¿Cuándo usarlo?

- Cuando necesites que el modelo aprenda de ejemplos concretos.
- Para preguntas que tienen respuestas definidas y repetitivas.



In [ ]:
from langchain import FewShotPromptTemplate

# Lista de ejemplos
ejemplos = [
    {"pregunta": "¿Cuál es el planeta más cercano al Sol?", "respuesta": "Mercurio."},
    {"pregunta": "¿Cuál es el río más largo del mundo?", "respuesta": "El Amazonas."},
]

promt_temp_ejemplos = PromptTemplate(input_variables=["pregunta", "respuesta"], 
                                     template="Pregunta: {pregunta}\nRespuesta: {respuesta}")

promt_ejemplos = FewShotPromptTemplate(
    example_prompt=promt_temp_ejemplos, 
    examples=ejemplos, 
    prefix="Eres un asistente que responde preguntas breves sobre datos curiosos.",
    suffix="Pregunta: {pregunta}\nRespuesta:", 
    input_variables=["pregunta"]
)

prompt_value = promt_ejemplos.format(pregunta="¿Cuál es el animal terrestre más rápido?")
respuesta_ejemplos = llm_openai(prompt_value)
print(respuesta_ejemplos)


### 6. Output Parser

Los parsers permiten transformar o estructurar las respuestas del modelo para que sean más útiles en tu aplicación.
¿Cuándo usar parsers?
- Si necesitas que las respuestas estén en un formato específico (listas, JSON, etc.).
- Para integrar la salida del modelo con otras aplicaciones o sistemas.

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate

output_parser = CommaSeparatedListOutputParser() #elijo el parseador que prefiera
format_instructions = output_parser.get_format_instructions()

template_basico_parser = """Cuales son los ingredientes para preparar {plato}\n{como_parsear}"""

prompt_temp_parser = PromptTemplate(
    input_variables=["plato"], 
    template=template_basico_parser, 
    partial_variables={"como_parsear": format_instructions}
)

promt_value_parser = prompt_temp_parser.format(plato="empanadas")
respuesta_parser = llm_openai(promt_value_parser) #se formula igual que antes y luego se imprime parseada
print(output_parser.parse(respuesta_parser))#aca imprime parseada
